# BIAFlows

## Nuclei Tracking 2D+t

### Fiji-workflow

The workflow treats the time-dimension as z-dimension and does a 3D-segmentation of the objects. The resulting object slices are then reduced to a center point.

#### Initialize the cytomine client and job

The cytomine client needs some parameters to connect to the server. The host on which the server is running, the public and private keys to make a connection, the id of the project and the id of the software.
The remaining 3 parameters will be passed on to the image analysis workflow.

In [ ]:
from getpass import getpass 

publicKey = getpass("Please enter the public key: ")
privateKey = getpass("Please enter the private key: ")

argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  'neubias.cytomine.be',
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  '1695226', 
 '--cytomine_id_software', '3804897',
 '--ij_gauss_radius',  '3',
 '--ij_threshold',  '60',
 '--ij_open_radius', '7']

Import CytomineJob and Job and update the status information. Set the problem class to particle-tracking (PrtTrk).

In [ ]:
import sys
from subprocess import call

from cytomine.models import Job

from neubiaswg5 import CLASS_PRTTRK
from neubiaswg5.helpers import NeubiasJob, prepare_data, upload_data, upload_metrics

jobID=-666
with NeubiasJob.from_cli(argv) as nj:
    nj.job.update(status=Job.RUNNING, progress=0, statusComment="Initialisation...")
    jobID = nj.job.id
    
problem_cls = CLASS_PRTTRK

#### Create local directories and download images

Create the local working directories in a subfolder jobID of the user's home folder.

- in: input images
- out: output images
- ground_truth: ground truth images
- tmp: temporary path

In [ ]:
in_images, gt_images, in_path, gt_path, out_path, tmp_path = prepare_data(problem_cls, nj, is_2d=False, **nj.flags)

#### Call the image analysis workflow 

In [ ]:
from subprocess import call

nj.job.update(progress=25, statusComment="Launching workflow...")
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro NucleiTracking.ijm " \
		"\"input={}, output={}, gauss_rad={}, threshold={}, open_rad={}\" -batch".format(in_path, out_path, nj.parameters.ij_gauss_radius, nj.parameters.ij_threshold, nj.parameters.ij_open_radius)

return_code = call(command, shell=True, cwd="./Fiji.app")  # waits for the subprocess to return

if return_code != 0:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    nj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)
nj.job.update(progress=30, statusComment="Workflow finished...")


#### Create and upload annotations

In [ ]:
from neubiaswg5.exporter.mask_to_points import mask_to_points_3d
import io
from skimage import io
from cytomine.models import Annotation,AnnotationCollection, ImageGroupCollection, ImageInstance, ImageSequenceCollection
from itertools import groupby
from shapely.geometry import LineString
from shapely.affinity import affine_transform
import numpy as np
   
for image in nj.monitor(input_images, start=60, end=80, period=0.1, prefix="Extracting and uploading tracks from point-masks"):
    file = "{}.tif".format(image.id)
    path = os.path.join(out_path, file)
    data = io.imread(path)

    # extract objects
    objects = mask_to_points_3d(data, time=True)

    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.time)    
    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.label)    
    grouped = groupby(objects,  key=lambda annotationSlice: annotationSlice.label)
    
    image_sequences = ImageSequenceCollection().fetch_with_filter("imagegroup", image.id)
    
    depth_to_image = {iseq.time: iseq.image for iseq in image_sequences}
    height = ImageInstance().fetch(image_sequences[0].image).height
    
    collection = AnnotationCollection()
    
    groups = []
    uniquekeys = []
    for k, g in grouped:
        groups.append(list(g))      # Store group iterator as a list
        uniquekeys.append(k)
    
    for key, group in zip(uniquekeys, groups):
        listOfPoints = []
        for oSlice in group:
            listOfPoints.append((oSlice.polygon.x, oSlice.polygon.y))
        shape = LineString(listOfPoints)
        for oSlice2 in group:
            polygon = affine_transform(shape, [1, 0, 0, -1, 0, height])
            annotation = Annotation(location=polygon.wkt, id_image=depth_to_image[oSlice2.time], id_project=cj.parameters.cytomine_id_project, property=[{"key": "index", "value": str(oSlice2.label)}])
            collection.append(annotation)
        
    collection.save()
    print("Found {} objects in this image {}.".format(len(objects), image.id))
    
    

#### Calculate and upload metrics

In [ ]:
# 5. Compute and upload the metrics
nj.job.update(progress=80, statusComment="Computing and uploading metrics (if necessary)...")
upload_metrics(problem_cls, nj, in_images, gt_path, out_path, tmp_path, **nj.flags)

nj.job.update(status=Job.TERMINATED, progress=100, statusComment="Finished.")

#### Cleanup - remove the files and folders that have been downloaded or created.

In [ ]:
for image in input_images:
    file = str(image.id)
    path = out_path + "/" + file + ".tif"
    os.remove(path);
    path = in_path + "/" + file  + ".tif"
    os.remove(path);
    path = gt_path + "/" + file  + ".tif"
    os.remove(path);
    
tmpData = ['intracks.xml', 'intracks.xml.score.txt', 'reftracks.xml']
for file in tmpData:
    path = tmp_path + "/" + file
    os.remove(path);
if os.path.exists(working_path):
    os.rmdir(in_path)
    os.rmdir(out_path)
    os.rmdir(gt_path)
    os.rmdir(tmp_path)
    os.rmdir(working_path)

#### End the job and the connection to cytomine

In [ ]:
nj.job.update(status=Job.TERMINATED, progress=100, statusComment="Finished.")
print("END.")